<a href="https://colab.research.google.com/github/reznimat/ppzd_sp/blob/main/semestralni_prace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import csv
import pandas as pd

url = "https://raw.githubusercontent.com/reznimat/ppzd_sp/main/Online_Sales.csv"
sales = pd.read_csv(url)

print(sales.count())

CustomerID;Transaction_ID;Transaction_Date;Product_SKU;Product_Description;Product_Category;Quantity;Avg_Price;Delivery_Charges;Coupon_Status    52924
dtype: int64
